In [25]:
import numpy as np
import pathlib
import xml.etree.ElementTree as ET
# from matplotlib import cm
import meshio
import trimesh
import igl
# from tqdm.notebook import tqdm
from tqdm import tqdm

In [26]:
def resolve_path(path, root: pathlib.Path) -> pathlib.Path:
    path = pathlib.Path(path)
    if path.is_absolute():
        return path
    return (root / path).resolve()


def parse_vtm(path):
    tree = ET.parse(path)
    root = tree.getroot()
    blocks = root.find("vtkMultiBlockDataSet").findall("Block")
    # assert(len(blocks) == 1)
    for block in blocks:
        # if block.get("name") == "Volume":
        if block.get("name") == "Surface":
            break
    datasets = block.findall("DataSet")
    for dataset in datasets:
        # if dataset.get("name") == "data":
        # if dataset.get("name") == "surface":
        if dataset.get("name") == "contact":
            break
    return resolve_path(dataset.attrib["file"], path.parent)


def parse_pvd(path):
    tree = ET.parse(path)
    root = tree.getroot()
    frames = root.find("Collection").findall("DataSet")
    assert(len(frames) > 0)

    meshes = []
    for f in map(lambda f: f.attrib["file"], frames):
        f = resolve_path(f, path.parent)
        if f.suffix == ".vtm":
            f = parse_vtm(f)
        meshes.append(resolve_path(f, path.parent))

    return meshes

In [27]:
def fix_normals(V, F):
    mesh = trimesh.Trimesh(V, F, process=False, validate=False)
    trimesh.repair.fix_normals(mesh)
    return mesh.faces


def load_mesh(path):
    mesh = meshio.read(path)

    V, I, J, _ = igl.remove_duplicate_vertices(
        mesh.points, np.array([], dtype=int), 1e-7)

    CV = []  # codim vertices
    E = []  # edges
    F = []  # triangles
    for cells in mesh.cells:
        if cells.type == "triangle":
            F.append(J[cells.data])
        elif cells.type == "tetra":
            F.append(fix_normals(V, igl.boundary_facets(J[cells.data])))
        elif cells.type == "line":
            E.append(J[cells.data])
        elif cells.type == "vertex":
            CV.append(J[cells.data])
        else:
            raise Exception("Unsupported cell type: {}".format(cells.type))

    cells = []
    if F:
        cells.append(("triangle", np.vstack(F).astype("int32")))
    if E:
        cells.append(("line", np.vstack(E).astype("int32")))
    if CV:
        cells.append(("vertex", np.vstack(CV).astype("int32")))

    if "solution" in mesh.point_data:
        V += mesh.point_data["solution"][I]

    point_data = {}
    # point_data = dict((k, v[I]) for k, v in mesh.point_data.items())
    # if "E" in point_data:
    #     point_data["is_obstacle"] = (point_data["E"] == 0).flatten()
    # else:
    #     point_data["is_obstacle"] = np.zeros((V.shape[0],), dtype=bool)

    mesh = meshio.Mesh(points=V, cells=cells, point_data=point_data)

    return mesh


In [28]:
sim_files = results = [[
    "../results/ball-wall/3D-noremesh-nref0/2023_04_09_17_19_21_476",
    "../results/ball-wall/3D-noremesh-nref1/2023_04_09_17_19_20_847",
    "../results/ball-wall/3D-noremesh-nref2/2023_04_09_17_19_20_806",
    "../results/ball-wall/3D-noremesh-nref3/2023_04_09_17_19_20_847",
    "../results/ball-wall/3D/2023_04_09_17_19_20_812",
    "../results/masticator/3D-noremesh-nref0/2023_04_09_17_19_20_759",
    "../results/masticator/3D-noremesh-nref1/2023_04_09_17_23_20_567",
    "../results/masticator/3D-noremesh-nref2/2023_04_09_18_20_29_496",
    "../results/masticator/3D-noremesh-nref3/2023_04_09_20_14_41_972",
    "../results/masticator/3D/2023_04_09_17_19_20_809",
    "../results/rollers/monkey-soft-hard-noremesh-nref0/2023_04_07_17_26_06_980",
    "../results/rollers/monkey-soft-hard-noremesh-nref1/2023_04_07_17_28_07_468",
    "../results/rollers/monkey-soft-hard-noremesh-nref2/2023_04_07_17_31_07_765",
    "../results/rollers/monkey-soft-hard-noremesh-nref3/2023_04_07_17_32_07_423",
    "../results/rollers/monkey-soft-hard/2023_04_10_20_22_50_759",
    "../results/spikes3d/restart_031-noremesh-nref0/2023_04_13_21_55_40_484",
    "../results/spikes3d/restart_031-noremesh-nref1/2023_04_13_21_55_40_512",
    "../results/spikes3d/restart_031-noremesh-nref2/2023_04_13_21_55_40_525",
    "../results/spikes3d/restart_031-noremesh-nref3/xxx",
    "../results/spikes3d/restart_031/2023_04_13_21_55_40_703",
    "../results/twisting-beam/twisting-beam-noremesh-nref0/2023_04_10_19_53_48_263",
    "../results/twisting-beam/twisting-beam-noremesh-nref1/2023_04_10_19_55_48_223",
    "../results/twisting-beam/twisting-beam-noremesh-nref2/2023_04_10_20_04_48_871",
    "../results/twisting-beam/twisting-beam-noremesh-nref3/2023_04_10_19_53_48_278",
    "../results/twisting-beam/twisting-beam/2023_04_10_19_53_48_283",
]]
sim_files = [pathlib.Path(f) for f in sim_files]

mesh_sequences = [{
    "name": pathlib.Path("."),
    "meshes": []
}]
for f in sim_files:
    if f.suffix == ".pvd":
        mesh_sequences.append({
            "name": f,
            "meshes": parse_pvd(f)
        })
    else:
        mesh_sequences[0]["meshes"].append(f)

for seq in mesh_sequences:
    if not seq["meshes"]:
        continue

    out_dir = seq["name"].parent / "plys"
    out_dir.mkdir(parents=True, exist_ok=True)

    for i, mesh_path in enumerate(tqdm(seq["meshes"])):
        mesh = load_mesh(mesh_path)
        meshio.write(out_dir / f"{i:04d}.ply", mesh)


100%|██████████| 2001/2001 [04:15<00:00,  7.83it/s]
